In [2]:
import pandas as pd
import numpy as np
import seaborn as sns
import os 
import matplotlib.pyplot as plt
import tqdm
import networkx as nx
import operator
from sqlalchemy import create_engine
import psycopg2
import os
import sys
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 50)

### Read the previously somewhat cleaned meta csv

In [ ]:
#read csv from sds project folder on rivanna and drop the unnamed column and duplicates becase every package irrespective of version will have same slug.
df=pd.read_csv('/project/class/sds_sdad/oss_capstone2021-2022/clean_data_11132021/pypi_meta_with_license_github_slugs.csv')\
.drop(columns=['Unnamed: 0']).drop_duplicates(['name'])

In [7]:
#drop nas so that filtering is easy
df=df.dropna()

In [20]:
#check rows
df.shape

(55370, 8)

### Cleaning github.com

In [16]:
#create new dataframe with unclean slugs, i.e, filtered on thsoe with github.com in the front
df_unclean = df[df.slug.str.contains('.*github.com/')]


In [21]:
#check rows
df_unclean.shape

(312, 8)

In [26]:
#remove unlean from original dataset so that we can append it back after we clean them
df = df[~df.name.isin(df_unclean.name)]

In [27]:
#check rows
df.shape

(55058, 8)

In [ ]:
#replace the github.com with ""
df_unclean.slug = df_unclean.slug.str.replace('.*github.com/', "")

In [30]:
#concat cleaned rows with original filtered df
df_clean = pd.concat([df,df_unclean])
df_clean.shape

(55370, 8)

### Cleaning right slashes

In [41]:
#make a df to now clean slashes at the end of string by using rstrip
df_no_slash = df_clean.copy()
df_no_slash.slug = df_no_slash.slug.str.rstrip('//')

### Make slugs for only usernames

In [51]:
#new df with no github.io and slashes at end
df_new_slugs = df_no_slash[~((df_no_slash.slug.str.contains('/')) | (df_no_slash.slug.str.contains('github.io')) )]

In [53]:
#create slugs for only usernames hopin it will be a slug
df_new_slugs.slug = df_new_slugs.slug+'/'+df_new_slugs.name

/apps/software/standard/compiler/gcc/9.2.0/jupyter_conda/2020.11-py3.8/lib/python3.8/site-packages/pandas/core/generic.py:5168: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[name] = value


In [54]:
#check shape
df_no_slash.shape

(55370, 8)

In [55]:
#cech shape of new slugs
df_new_slugs.shape

(585, 8)

In [56]:
#check rows on filtered dataframe
df_no_slash = df_no_slash[~df_no_slash.name.isin(df_new_slugs.name)]
df_no_slash.shape

(54785, 8)

In [57]:
#concatenate both
df_final = pd.concat([df_no_slash,df_new_slugs])
df_final.shape

(55370, 8)

### save as csv

In [61]:
SAve as csv
df_final.to_csv('oss_pypi_2021_cleaner_slugs_02122021')